In [ ]:
import sys
sys.path.append('..')
from datetime import datetime
from data.random_data import ClimatehackDataset as Dataset
from submission.keys import META as META_KEYS, WEATHER as WEATHER_KEYS, NONHRV as NONHRV_KEYS

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ds = Dataset(
    start_date=datetime(2020, 1, 1),
    end_date=datetime(2020, 1, 2),
    root_dir='/data/climatehack/',
    meta_features=META_KEYS,
    nonhrv_features=NONHRV_KEYS,
    weather_features=WEATHER_KEYS,
    future_features=[]
)

In [ ]:
i = np.random.randint(0, len(ds))
pv, meta, imgs, weather, target = ds[i]
imgs = imgs[NONHRV_KEYS.VIS006]
a, b, c = imgs[0].numpy(), imgs[1].numpy(), imgs[11].numpy()
flow = cv2.calcOpticalFlowFarneback(
    prev=a,
    next=b,
    flow=None,
    pyr_scale=0.5,
    levels=2,
    winsize=40,
    iterations=3,
    poly_n=5,
    poly_sigma=0.7,
    flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN
)
flow = -flow

# x = (all_data[:INPUT_STEPS] - _MEAN_PIXEL) / _STD_PIXEL
# y = all_data[INPUT_STEPS:]

# flow_x = (flow[:, :, 0] - _MEAN_X_FLOW) / _STD_X_FLOW
# flow_y = (flow[:, :, 1] - _MEAN_Y_FLOW) / _STD_Y_FLOW

# flow_x = torch.FloatTensor(flow_x)
# flow_y = torch.FloatTensor(flow_y)

# x = torch.cat(
#     [
#         x,
#         osgb_x.unsqueeze(0),
#         osgb_y.unsqueeze(0),
#         flow_x.unsqueeze(0),
#         flow_y.unsqueeze(0),
#     ],
#     dim=0,
# )
# return x, y

map = np.zeros_like(flow)
for y in range(map.shape[0]):
    for x in range(map.shape[1]):
        f = 10 * flow[y, x]
        map[y, x] = (x + f[0], y + f[1])

out = cv2.remap(b, map, None, cv2.INTER_LINEAR)

In [ ]:
f, ax = plt.subplots(1, 4, figsize=(20, 8))
ax[0].imshow(a, cmap='gray')
ax[1].imshow(b, cmap='gray')
ax[2].imshow(c, cmap='gray')
ax[3].imshow(out, cmap='gray')